In [5]:
try:
    import elasticsearch
    from elasticsearch import Elasticsearch
    
    import pandas as pd
    import json
    from ast import literal_eval
    from tqdm import tqdm
    import datetime
    import os
    import sys
    
    import os
    import numpy as np
    import tensorflow as tf
    
    from elasticsearch import helpers
    
    from gensim.models import Word2Vec, KeyedVectors
    
    import nltk
    import re
    from nltk.corpus import stopwords
    nltk.download('stopwords')
    
    import spacy
    import pickle

    print("Loaded  .. . . . . . . .")
except Exception as E:
    print("Some Modules are Missing {} ".format(E))

Loaded  .. . . . . . . .


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/srinivas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df = pd.read_csv("val_knowledgebase.csv")
print("read")

read


In [7]:
df.head()

,atom,country,product,query
0,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),What is a calcium ion influx inhibitor?
1,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),What class of calcium channel blockers is Felo...
2,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),Felodipine is a member of which class of inhib...
3,The therapeutic effect of this group of drugs ...,Canada,Plendil(Felodipine),What is the therapeutic effect of Felodipine r...
4,The therapeutic effect of this group of drugs ...,Canada,Plendil(Felodipine),How are the contractile processes of these tis...


In [8]:
country=df["country"]
country_set=set(country)
product=df["product"]
product_set=set(product)
print("countries : "+str(country_set))
print("products : "+str(product_set))

countries : {'United States', 'Canada', 'European Union'}
products : {'Plendil(Felodipine)', 'Rosuvastatin(Crestor)', 'Zestril (lisinopril)'}


In [10]:
with open('/home/srinivas/Desktop/validation_arrays/fast/fasttext_answer.pkl','rb') as f:
     fast_answer = pickle.load(f)
df["fasttext_answer"]=fast_answer

In [11]:
with open('/home/srinivas/Desktop/validation_arrays/fast/fasttext_question.pkl','rb') as f:
     fast_question = pickle.load(f)
df["fasttext_question"]=fast_question

In [12]:
df.head()

,atom,country,product,query,fasttext_answer,fasttext_question
0,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),What is a calcium ion influx inhibitor?,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.11014999728649855, 0.007775001227855682, 0..."
1,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),What class of calcium channel blockers is Felo...,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.10943333432078362, 0.006266666886707147, 0..."
2,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),Felodipine is a member of which class of inhib...,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.1116000022739172, -0.0011249997187405825, ..."
3,The therapeutic effect of this group of drugs ...,Canada,Plendil(Felodipine),What is the therapeutic effect of Felodipine r...,"[-0.051349122751210084, 0.0010666667520509738,...","[0.000200001522898674, 0.0321500007994473, 0.1..."
4,The therapeutic effect of this group of drugs ...,Canada,Plendil(Felodipine),How are the contractile processes of these tis...,"[-0.051349122751210084, 0.0010666667520509738,...","[-0.02942222191227807, 0.038144443132397204, 0..."


In [13]:
with open('/home/srinivas/Desktop/validation_arrays/w2v/w2v_question.pkl','rb') as f:
     w2v_ques = pickle.load(f)
df["w2v_question"]=w2v_ques

In [14]:
with open('/home/srinivas/Desktop/validation_arrays/w2v/w2v_answer.pkl','rb') as f:
     w2v_ans = pickle.load(f)
df["w2v_answer"]=w2v_ans

In [24]:
type(w2v_ans[0])

numpy.ndarray

In [15]:
df.head()

,atom,country,product,query,fasttext_answer,fasttext_question,w2v_question,w2v_answer
0,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),What is a calcium ion influx inhibitor?,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.11014999728649855, 0.007775001227855682, 0...","[-0.06475830078125, 0.20947265625, -0.01521396...","[-0.10657552083333334, 0.069140625, 0.02999191..."
1,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),What class of calcium channel blockers is Felo...,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.10943333432078362, 0.006266666886707147, 0...","[-0.09993489583333333, 0.004964192708333333, 0...","[-0.10657552083333334, 0.069140625, 0.02999191..."
2,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),Felodipine is a member of which class of inhib...,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.1116000022739172, -0.0011249997187405825, ...","[-0.037353515625, 0.00390625, 0.048828125, 0.1...","[-0.10657552083333334, 0.069140625, 0.02999191..."
3,The therapeutic effect of this group of drugs ...,Canada,Plendil(Felodipine),What is the therapeutic effect of Felodipine r...,"[-0.051349122751210084, 0.0010666667520509738,...","[0.000200001522898674, 0.0321500007994473, 0.1...","[-0.06671142578125, 0.0811767578125, 0.0469970...","[-0.04052091899671053, 0.09128396552905702, 0...."
4,The therapeutic effect of this group of drugs ...,Canada,Plendil(Felodipine),How are the contractile processes of these tis...,"[-0.051349122751210084, 0.0010666667520509738,...","[-0.02942222191227807, 0.038144443132397204, 0...","[-0.03811306423611111, 0.14531114366319445, 0....","[-0.04052091899671053, 0.09128396552905702, 0...."


In [22]:

with open('/home/srinivas/Desktop/validation_arrays/bert/bert_answer.pkl','rb') as f:
     x77 = pickle.load(f)
type(x77)

numpy.ndarray

In [26]:
bert_answer_list=[]
for x in x77:
    bert_answer_list.append(x)
df["bert_answer"]=bert_answer_list

In [27]:
with open('/home/srinivas/Desktop/validation_arrays/bert/bert_question.pkl','rb') as f:
     x77a = pickle.load(f)
type(x77a)

numpy.ndarray

In [28]:
bert_question_list=[]
for x in x77a:
    bert_question_list.append(x)
df["bert_question"]=bert_question_list

In [29]:
df.head()

,atom,country,product,query,fasttext_answer,fasttext_question,w2v_question,w2v_answer,bert_answer,bert_question
0,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),What is a calcium ion influx inhibitor?,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.11014999728649855, 0.007775001227855682, 0...","[-0.06475830078125, 0.20947265625, -0.01521396...","[-0.10657552083333334, 0.069140625, 0.02999191...","[-0.1021475, 0.19372933, -0.21815854, 0.206826...","[0.2899235, -0.121713676, 0.24831414, 0.076035..."
1,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),What class of calcium channel blockers is Felo...,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.10943333432078362, 0.006266666886707147, 0...","[-0.09993489583333333, 0.004964192708333333, 0...","[-0.10657552083333334, 0.069140625, 0.02999191...","[-0.1021475, 0.19372933, -0.21815854, 0.206826...","[-0.04205227, -0.019552507, 0.16107942, 0.3579..."
2,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),Felodipine is a member of which class of inhib...,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.1116000022739172, -0.0011249997187405825, ...","[-0.037353515625, 0.00390625, 0.048828125, 0.1...","[-0.10657552083333334, 0.069140625, 0.02999191...","[-0.1021475, 0.19372933, -0.21815854, 0.206826...","[0.37489668, 0.033670336, -0.09372824, 0.08862..."
3,The therapeutic effect of this group of drugs ...,Canada,Plendil(Felodipine),What is the therapeutic effect of Felodipine r...,"[-0.051349122751210084, 0.0010666667520509738,...","[0.000200001522898674, 0.0321500007994473, 0.1...","[-0.06671142578125, 0.0811767578125, 0.0469970...","[-0.04052091899671053, 0.09128396552905702, 0....","[-0.07960795, 0.12230982, -0.2276642, -0.00896...","[0.050525643, 0.058479086, 0.17381626, 0.10562..."
4,The therapeutic effect of this group of drugs ...,Canada,Plendil(Felodipine),How are the contractile processes of these tis...,"[-0.051349122751210084, 0.0010666667520509738,...","[-0.02942222191227807, 0.038144443132397204, 0...","[-0.03811306423611111, 0.14531114366319445, 0....","[-0.04052091899671053, 0.09128396552905702, 0....","[-0.07960795, 0.12230982, -0.2276642, -0.00896...","[-0.14897719, 0.051382728, -0.06139521, 0.1016..."


In [30]:
with open('/home/srinivas/Desktop/validation_arrays/spacy/spacy_question.pkl','rb') as f:
     spacy_ques = pickle.load(f)
df["spacy_question"]=spacy_ques

In [31]:
with open('/home/srinivas/Desktop/validation_arrays/spacy/spacy_answer.pkl','rb') as f:
     spacy_ans = pickle.load(f)
df["spacy_answer"]=spacy_ans

In [32]:
df.head()

,atom,country,product,query,fasttext_answer,fasttext_question,w2v_question,w2v_answer,bert_answer,bert_question,spacy_question,spacy_answer
0,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),What is a calcium ion influx inhibitor?,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.11014999728649855, 0.007775001227855682, 0...","[-0.06475830078125, 0.20947265625, -0.01521396...","[-0.10657552083333334, 0.069140625, 0.02999191...","[-0.1021475, 0.19372933, -0.21815854, 0.206826...","[0.2899235, -0.121713676, 0.24831414, 0.076035...","[-0.24090976, 0.256362, 0.020495497, -0.099701...","[-0.088737875, 0.30348238, -0.027731512, -0.03..."
1,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),What class of calcium channel blockers is Felo...,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.10943333432078362, 0.006266666886707147, 0...","[-0.09993489583333333, 0.004964192708333333, 0...","[-0.10657552083333334, 0.069140625, 0.02999191...","[-0.1021475, 0.19372933, -0.21815854, 0.206826...","[-0.04205227, -0.019552507, 0.16107942, 0.3579...","[-0.012048334, 0.32869837, -0.0176391, -0.0020...","[-0.088737875, 0.30348238, -0.027731512, -0.03..."
2,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),Felodipine is a member of which class of inhib...,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.1116000022739172, -0.0011249997187405825, ...","[-0.037353515625, 0.00390625, 0.048828125, 0.1...","[-0.10657552083333334, 0.069140625, 0.02999191...","[-0.1021475, 0.19372933, -0.21815854, 0.206826...","[0.37489668, 0.033670336, -0.09372824, 0.08862...","[0.028587498, 0.0662625, 0.1000235, 0.00710500...","[-0.088737875, 0.30348238, -0.027731512, -0.03..."
3,The therapeutic effect of this group of drugs ...,Canada,Plendil(Felodipine),What is the therapeutic effect of Felodipine r...,"[-0.051349122751210084, 0.0010666667520509738,...","[0.000200001522898674, 0.0321500007994473, 0.1...","[-0.06671142578125, 0.0811767578125, 0.0469970...","[-0.04052091899671053, 0.09128396552905702, 0....","[-0.07960795, 0.12230982, -0.2276642, -0.00896...","[0.050525643, 0.058479086, 0.17381626, 0.10562...","[-0.0468025, 0.19192249, -0.19600251, -0.04838...","[-0.072944276, 0.2486027, -0.19252801, 0.05586..."
4,The therapeutic effect of this group of drugs ...,Canada,Plendil(Felodipine),How are the contractile processes of these tis...,"[-0.051349122751210084, 0.0010666667520509738,...","[-0.02942222191227807, 0.038144443132397204, 0...","[-0.03811306423611111, 0.14531114366319445, 0....","[-0.04052091899671053, 0.09128396552905702, 0....","[-0.07960795, 0.12230982, -0.2276642, -0.00896...","[-0.14897719, 0.051382728, -0.06139521, 0.1016...","[-0.13795668, 0.115536675, -0.18193598, 0.0613...","[-0.072944276, 0.2486027, -0.19252801, 0.05586..."


In [33]:
with open('/home/srinivas/Desktop/validation_arrays/use/universal_question_series.pkl','rb') as f:
     use_ques = pickle.load(f)
df["use_question"]=use_ques

In [34]:
with open('/home/srinivas/Desktop/validation_arrays/use/universal_answer_series.pkl','rb') as f:
     use_ans = pickle.load(f)
df["use_answer"]=use_ans

In [35]:
df.head()

,atom,country,product,query,fasttext_answer,fasttext_question,w2v_question,w2v_answer,bert_answer,bert_question,spacy_question,spacy_answer,use_question,use_answer
0,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),What is a calcium ion influx inhibitor?,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.11014999728649855, 0.007775001227855682, 0...","[-0.06475830078125, 0.20947265625, -0.01521396...","[-0.10657552083333334, 0.069140625, 0.02999191...","[-0.1021475, 0.19372933, -0.21815854, 0.206826...","[0.2899235, -0.121713676, 0.24831414, 0.076035...","[-0.24090976, 0.256362, 0.020495497, -0.099701...","[-0.088737875, 0.30348238, -0.027731512, -0.03...","[0.027756759896874428, 0.007806204259395599, -...","[0.04650423303246498, 0.03865881636738777, -0...."
1,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),What class of calcium channel blockers is Felo...,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.10943333432078362, 0.006266666886707147, 0...","[-0.09993489583333333, 0.004964192708333333, 0...","[-0.10657552083333334, 0.069140625, 0.02999191...","[-0.1021475, 0.19372933, -0.21815854, 0.206826...","[-0.04205227, -0.019552507, 0.16107942, 0.3579...","[-0.012048334, 0.32869837, -0.0176391, -0.0020...","[-0.088737875, 0.30348238, -0.027731512, -0.03...","[0.0189292561262846, -0.021205231547355652, -0...","[0.04650423303246498, 0.03865881636738777, -0...."
2,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),Felodipine is a member of which class of inhib...,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.1116000022739172, -0.0011249997187405825, ...","[-0.037353515625, 0.00390625, 0.048828125, 0.1...","[-0.10657552083333334, 0.069140625, 0.02999191...","[-0.1021475, 0.19372933, -0.21815854, 0.206826...","[0.37489668, 0.033670336, -0.09372824, 0.08862...","[0.028587498, 0.0662625, 0.1000235, 0.00710500...","[-0.088737875, 0.30348238, -0.027731512, -0.03...","[0.02716185711324215, 0.0028547781985253096, -...","[0.04650423303246498, 0.03865881636738777, -0...."
3,The therapeutic effect of this group of drugs ...,Canada,Plendil(Felodipine),What is the therapeutic effect of Felodipine r...,"[-0.051349122751210084, 0.0010666667520509738,...","[0.000200001522898674, 0.0321500007994473, 0.1...","[-0.06671142578125, 0.0811767578125, 0.0469970...","[-0.04052091899671053, 0.09128396552905702, 0....","[-0.07960795, 0.12230982, -0.2276642, -0.00896...","[0.050525643, 0.058479086, 0.17381626, 0.10562...","[-0.0468025, 0.19192249, -0.19600251, -0.04838...","[-0.072944276, 0.2486027, -0.19252801, 0.05586...","[0.04280677065253258, 0.031687188893556595, -0...","[0.040527839213609695, 0.058647628873586655, -..."
4,The therapeutic effect of this group of drugs ...,Canada,Plendil(Felodipine),How are the contractile processes of these tis...,"[-0.051349122751210084, 0.0010666667520509738,...","[-0.02942222191227807, 0.038144443132397204, 0...","[-0.03811306423611111, 0.14531114366319445, 0....","[-0.04052091899671053, 0.09128396552905702, 0....","[-0.07960795, 0.12230982, -0.2276642, -0.00896...","[-0.14897719, 0.051382728, -0.06139521, 0.1016...","[-0.13795668, 0.115536675, -0.18193598, 0.0613...","[-0.072944276, 0.2486027, -0.19252801, 0.05586...","[0.003646381665021181, 0.06274382770061493, -0...","[0.040527839213609695, 0.058647628873586655, -..."


In [ ]:
print(len(spacy_vec))
print(len(fast_vec))
print(len(w2v_vec))

In [36]:
df.to_pickle("/home/srinivas/Desktop/dataframe/the_data.pkl")

In [37]:
with open("/home/srinivas/Desktop/dataframe/the_data.pkl",'rb') as f:
     success= pickle.load(f)


In [38]:
success

,atom,country,product,query,fasttext_answer,fasttext_question,w2v_question,w2v_answer,bert_answer,bert_question,spacy_question,spacy_answer,use_question,use_answer
0,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),What is a calcium ion influx inhibitor?,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.11014999728649855, 0.007775001227855682, 0...","[-0.06475830078125, 0.20947265625, -0.01521396...","[-0.10657552083333334, 0.069140625, 0.02999191...","[-0.1021475, 0.19372933, -0.21815854, 0.206826...","[0.2899235, -0.121713676, 0.24831414, 0.076035...","[-0.24090976, 0.256362, 0.020495497, -0.099701...","[-0.088737875, 0.30348238, -0.027731512, -0.03...","[0.027756759896874428, 0.007806204259395599, -...","[0.04650423303246498, 0.03865881636738777, -0...."
1,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),What class of calcium channel blockers is Felo...,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.10943333432078362, 0.006266666886707147, 0...","[-0.09993489583333333, 0.004964192708333333, 0...","[-0.10657552083333334, 0.069140625, 0.02999191...","[-0.1021475, 0.19372933, -0.21815854, 0.206826...","[-0.04205227, -0.019552507, 0.16107942, 0.3579...","[-0.012048334, 0.32869837, -0.0176391, -0.0020...","[-0.088737875, 0.30348238, -0.027731512, -0.03...","[0.0189292561262846, -0.021205231547355652, -0...","[0.04650423303246498, 0.03865881636738777, -0...."
2,PLENDIL (felodipine) is a calcium ion influx i...,Canada,Plendil(Felodipine),Felodipine is a member of which class of inhib...,"[-0.11925999894738197, -0.009213333328564962, ...","[-0.1116000022739172, -0.0011249997187405825, ...","[-0.037353515625, 0.00390625, 0.048828125, 0.1...","[-0.10657552083333334, 0.069140625, 0.02999191...","[-0.1021475, 0.19372933, -0.21815854, 0.206826...","[0.37489668, 0.033670336, -0.09372824, 0.08862...","[0.028587498, 0.0662625, 0.1000235, 0.00710500...","[-0.088737875, 0.30348238, -0.027731512, -0.03...","[0.02716185711324215, 0.0028547781985253096, -...","[0.04650423303246498, 0.03865881636738777, -0...."
3,The therapeutic effect of this group of drugs ...,Canada,Plendil(Felodipine),What is the therapeutic effect of Felodipine r...,"[-0.051349122751210084, 0.0010666667520509738,...","[0.000200001522898674, 0.0321500007994473, 0.1...","[-0.06671142578125, 0.0811767578125, 0.0469970...","[-0.04052091899671053, 0.09128396552905702, 0....","[-0.07960795, 0.12230982, -0.2276642, -0.00896...","[0.050525643, 0.058479086, 0.17381626, 0.10562...","[-0.0468025, 0.19192249, -0.19600251, -0.04838...","[-0.072944276, 0.2486027, -0.19252801, 0.05586...","[0.04280677065253258, 0.031687188893556595, -0...","[0.040527839213609695, 0.058647628873586655, -..."
4,The therapeutic effect of this group of drugs ...,Canada,Plendil(Felodipine),How are the contractile processes of these tis...,"[-0.051349122751210084, 0.0010666667520509738,...","[-0.02942222191227807, 0.038144443132397204, 0...","[-0.03811306423611111, 0.14531114366319445, 0....","[-0.04052091899671053, 0.09128396552905702, 0....","[-0.07960795, 0.12230982, -0.2276642, -0.00896...","[-0.14897719, 0.051382728, -0.06139521, 0.1016...","[-0.13795668, 0.115536675, -0.18193598, 0.0613...","[-0.072944276, 0.2486027, -0.19252801, 0.05586...","[0.003646381665021181, 0.06274382770061493, -0...","[0.040527839213609695, 0.058647628873586655, -..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2250,Description:\tLisinopril is a white to off-whi...,Canada,Zestril (lisinopril),Lisinospril is what kind of powder?,"[-0.0022642851940223147, -0.0563214278081432, ...","[-0.0706000030040741, -0.026100000366568565, 0...","[0.05889892578125, -0.0111083984375, 0.1536865...","[0.026881626674107144, 0.005759102957589286, 0...","[-0.02737657, -0.07943635, -0.22290449, 0.0618...","[-0.044409927, -0.16870317, -0.15328597, 0.170...","[-0.07918367, 0.24674332, 0.029136, -0.0801326...","[-0.28126448, 0.22462204, -0.064237304, -0.127...","[-0.0

In [ ]:
print(len(spacy_vec[0]))
print(len(fast_vec[0]))
print(len(w2v_vec[0]))

In [ ]:
df["fasttext"]=fast_vec
df["w2v"]=w2v_vec
df["spacy"]=spacy_vec

In [ ]:
df.head()

In [ ]:
df.to_pickle("./validation_data_with_3_w2v_vectors.pkl")

In [ ]:
print(tf.__version__)

In [ ]:
Settings = {
   "settings":{
      "number_of_shards":1,
      "number_of_replicas":0
   },
   "mappings":{
       "dynamic": "true",
        "_source": {
        "enabled": "true"
        },
       "properties":{
            "answer": {
            "type": "text"
            },
          "fasttext_vector":{
         "type":"dense_vector",
         "dims":300
      },
        "w2v_vector":{
         "type":"dense_vector",
         "dims":300
      },
        "spacy_vector":{
         "type":"dense_vector",
         "dims":300
      },
          "country":{  
            "type":"text"
         },   
          "product":{  
            "type":"text"
         }
    }
   }
}

In [ ]:
ENDPOINT = "http://localhost:9200/"
es = Elasticsearch(timeout=600,hosts=ENDPOINT)
print("python client created")

In [ ]:
#creating the database
IndexName = 'ensemble_w2v_models'
my = es.indices.create(index=IndexName, ignore=[400,404], body=Settings)

In [ ]:
my

In [ ]:
df.columns

In [ ]:
df22 = df.to_dict('records')
df22

In [ ]:
len(df22)

In [ ]:
arr=[]
for c, line in enumerate(df22):
    arr.append({"answer":line.get("atom",""),
    'fasttext_vector':line.get("fasttext",""),
    'w2v_vector':line.get("w2v",""),
    'spacy_vector':line.get("spacy",""),
    'country':line.get("country","").lower().strip(),
    'product':line.get("product","").lower().strip()})

In [ ]:
arr

In [ ]:
for i in range(0,len(arr)):

    resp=es.index(index=IndexName,doc_type="_doc",id=i,body=arr[i])

In [ ]:
resp

In [ ]:
que="what is Crestor used for in canada?"
print(que)
temp_prod=df["product"]
temp_country=df["country"]
product_list=list(set(temp_prod))
print(product_list)
country_list=list(set(temp_country))
print(country_list)

In [ ]:
w2v_1=single_w2v(que)


In [ ]:
def list_generator():    
    temp_prod=df["product"]
    temp_country=df["country"]

    product_list=list(set(temp_prod))
    country_list=list(set(temp_country))

    for i in range(0,len(product_list)):
        product_list[i]=product_list[i].lower()

    for i in range(0,len(country_list)):
        country_list[i]=country_list[i].lower().strip()

    prods=[]

    for x in product_list:
        temp=x.split("(")
        for i in range(0,len(temp)):
            temp[i]=temp[i].replace(")","")
            temp[i]=temp[i].strip()
            prods.append(temp[i])
    return [prods,country_list]

    

In [ ]:
list_generator()

In [ ]:
def query_creator(que,lists):    
    prods=lists[0]
    country_list=lists[1]
    print(prods)
    print(country_list)
    
    spacy_vec=single_spacy(que)
    w2v_vec=single_w2v(que)
    fasttext_vec=single_fasttext(que)
    #print(que_vec)
    #print(que)
    que=que.lower()
    p=""
    c=""
    for x in prods:
        if x in que:
            p=x
            print(p)
            break
    for x in country_list:
        if x in que:
            c=x
            print(c)

    script_query={}
    if p!="" and c!="":
        print("1...........")
        script_query = {
        "script_score": {
        "query": {
    "bool": {
      "must": [
        {
          "match": {
            "product": p
          }
        }
      ],
      "should": [
        {
          "match": {
            "country": c
          }
        }
      ]
    }
  },
        "script": {
        "source": "((Math.abs(cosineSimilarity(params.spacy_vector, doc['spacy_vector'])))+(Math.abs(cosineSimilarity(params.w2v_vector, doc['w2v_vector'])))+(Math.abs(cosineSimilarity(params.fasttext_vector, doc['fasttext_vector']))))/3",
        "params": {"spacy_vector": spacy_vec,"w2v_vector": w2v_vec,"fasttext_vector": fasttext_vec}
        }}}

    elif p!="" and c=="":
        print("2...........")
        script_query = {
        "script_score": {
        "query": {"match": {
              "product": {
                "query":p
              }}},
        "script": {
        "source": "((Math.abs(cosineSimilarity(params.spacy_vector, doc['spacy_vector'])))+(Math.abs(cosineSimilarity(params.w2v_vector, doc['w2v_vector'])))+(Math.abs(cosineSimilarity(params.fasttext_vector, doc['fasttext_vector']))))/3",
        "params": {"spacy_vector": spacy_vec,"w2v_vector": w2v_vec,"fasttext_vector": fasttext_vec}
        }}}
    elif p=="" and c!="":
        print("3...........")
        script_query = {
        "script_score": {
        "query": {"match": {
              "country": {
                "query":c
              }}},
        "script": {
        "source": "((Math.abs(cosineSimilarity(params.spacy_vector, doc['spacy_vector'])))+(Math.abs(cosineSimilarity(params.w2v_vector, doc['w2v_vector'])))+(Math.abs(cosineSimilarity(params.fasttext_vector, doc['fasttext_vector']))))/3",
        "params": {"spacy_vector": spacy_vec,"w2v_vector": w2v_vec,"fasttext_vector": fasttext_vec}
        }}}
    else:
        print("4...........")
        script_query = {
        "script_score": {
        "query": {"match_all": {}},
        "script": {
        "source": "((Math.abs(cosineSimilarity(params.spacy_vector, doc['spacy_vector'])))+(Math.abs(cosineSimilarity(params.w2v_vector, doc['w2v_vector'])))+(Math.abs(cosineSimilarity(params.fasttext_vector, doc['fasttext_vector']))))/3",
        "params": {"spacy_vector": spacy_vec,"w2v_vector": w2v_vec,"fasttext_vector": fasttext_vec}
        }}}
    return  script_query

In [ ]:
lists=list_generator()

In [ ]:
lists

In [ ]:
que="what is Crestor used for ?"

In [ ]:
script_query=query_creator(que,lists)
script_query

In [ ]:
ans=es.search(index=IndexName,body={"size": 10,"query": script_query,"_source": {"includes": ["answer","country","product"]}})

In [ ]:
ans

In [ ]:
def questions():
    count=0
    que_arr=[]
    for x in df["query"]:
        sent=x.split("\n")
        for q in sent:
            if re.search('[a-zA-Z]',q)!=None:
                que_arr.append([q,count])
        count+=1
    return que_arr

In [ ]:
que_arr=questions()

In [ ]:
len(que_arr)

In [ ]:
que_arr

In [ ]:

def rank(ans):
    required=ans['hits']
    main_arr=required["hits"]
    count=1
    ids__=dict()
    for x in main_arr:
        id_no= x["_id"]

        score=x["_score"]

        answer_=x["_source"]["answer"]
        
        ids__[id_no.strip()]=1/count
        count+=1
    return ids__
    

In [ ]:
ranker=rank(ans)

In [ ]:
ranker

In [ ]:
ans=es.search(index=IndexName,body={"size": 10,"query": script_query,"_source": {"includes": ["answer"]}})

In [ ]:
def mrr():
    total=0
    count=0
    recall_count=0
    lists_all=list_generator()
    print(lists_all)
    for x in que_arr:
        question_=x[0].strip()
        print(question_)
        id_number=str(x[1])

        script_query_curr=query_creator(question_,lists_all)
        ans=es.search(index=IndexName,body={"size": 10,"query": script_query_curr,"_source": {"includes": ["answer","product","country"]}})
        ranker=rank(ans)
        if id_number in ranker:
            total+=ranker[id_number]
            print(ranker[id_number])
            recall_count+=1
        else:
            print(0)
        count+=1

    print("mrr is"+str(total/count))
    print("recall is"+str(recall_count/count))
    return
        

In [ ]:
mrr()